### pip install & import

In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import pandas as pd
#import pandas_profiling
import datetime

# 예지언니 크롤링 데이터 머지

### 데이터 불러오기

In [ ]:
movie_resize2 = pd.read_csv('../2_resizing/resizing.csv',encoding='utf-8')

### 크롤링된 파일 불러오기

In [ ]:
# 네이버 평점과 배우 가져옴

df2 = pd.read_csv("../../data/movie_data/concat.csv", encoding='UTF-8',index_col=0)
df2 = df2.drop(['네이버URL','이미지URL','제작연도','감독','연도'],axis=1)

### 빈칸 있는 데이터 삭제

In [ ]:
# null 값이 있는 열 삭제 
df2 = df2[df2['출연'].isnull() == False] 
df2.info()

### movie_resize 데이터 + 크롤링된 데이터 Merge

In [ ]:
movie_resize4 = pd.merge(movie_resize2,df2,on='영화명')

### 중복 데이터 삭제

In [ ]:
movie_resize4.drop_duplicates('영화명',keep='first',inplace=True)
movie_resize4.info()

In [ ]:
# Unnamed: 0 삭제하는 법
# drop 할 때에는 list에 넣어서 한다.

movie_resize4.drop(['Unnamed: 0'],axis=1,inplace=True)

In [ ]:
# 중복 데이터 없어졌는 지 확인
movie_resize4[movie_resize4.duplicated()==True]

In [ ]:
# 변경 될때만 따로 풀어서 사용하시길.(파일생성코드)
# movie_resize4.to_csv('./movie_resize_merged.csv',encoding='utf-8')

# movie_resize_merged.csv에서 top 300 생성

### Merge된 데이터 불러오기

In [2]:
movie_resize2 = pd.read_csv('./movie_resize_merged.csv',encoding='utf-8')

### Top 300 저장 및 불러오기

In [3]:
# 변경 될때만 따로 풀어서 사용하시길.(파일생성코드)
# col = ['영화명', '감독', '배급사', '개봉일', '출연']
# movie_resize2.reset_index(drop=False,inplace=True)
# movie_resize2[col][:300].to_csv(
#     './movie_resize_ranking.csv', encoding="utf-8")


In [4]:
# top 300 불러오기

movie_resize_ranking = pd.read_csv('./movie_resize_ranking.csv',encoding='UTF-8')
movie_resize_ranking.drop(['Unnamed: 0'], axis=1, inplace=True)
movie_resize_ranking

,영화명,감독,배급사,개봉일,출연
0,아바타: 물의 길,제임스 카메론,월트디즈니스튜디오스코리아,2022-12-14,"['시고니 위버', '스티븐 랭', '조 샐다나', '샘 워싱턴', '우나 채플린'..."
1,어벤져스: 에이지 오브 울트론,조스 웨던,월트디즈니스튜디오스코리아,2015-04-23,"['로버트 다우니 주니어', '크리스 헴스워스', '마크 러팔로', '크리스 에반스..."
2,기생충,봉준호,씨제이이엔엠,2019-05-30,"['송강호', '이선균', '조여정', '최우식', '박소담', '이정은', '장혜진']"
3,겨울왕국,제니퍼 리,월트디즈니스튜디오스코리아,2014-01-16,"['크리스틴 벨', '이디나 멘젤']"
4,인터스텔라,크리스토퍼 놀란,워너브러더스 코리아,2014-11-06,"['매튜 맥커너히', '앤 해서웨이', '마이클 케인', '제시카 차스테인']"
...,...,...,...,...,...
295,황해,나홍진,쇼박스,2010-12-22,"['하정우', '김윤석', '조성하']"
296,원더 우먼,팻티 젠킨스,워너브러더스 코리아,2017-05-31,"['갤 가돗', '크리스 파인']"
297,초능력자,김민석,넥스트엔터테인먼트월드,2010-11-10,"['강동원', '고수', '정은채']"
298,내 사랑 내 곁에,박진표,씨제이이엔엠,2009-09-24,"['하지원', '김명민']"


In [5]:
# 점수화를 위해 movie_resize_ranking['개봉일']을 datetime 타입으로 변경

movie_resize_ranking['개봉일'] = movie_resize_ranking['개봉일'].astype('datetime64')

### 점수화행 만들기

In [6]:
# 점수화행 만들기
movie_resize2['개봉일'] = movie_resize2['개봉일'].astype('datetime64')

movie_final = movie_resize2.copy()
movie_final['감독_흥행'] = [0 for _ in range(len(movie_resize2))]
movie_final['배급사_흥행'] = [0 for _ in range(len(movie_resize2))]
movie_final['주연배우_흥행'] = [0 for _ in range(len(movie_resize2))]

In [7]:
# 순번을 쓰기 위해 인덱스 풀기

movie_resize_ranking.reset_index(drop=False,inplace=True)

### 감독, 배급사 점수화 코드

In [8]:
# 감독 점수화 코드

for idx, row in movie_final.iterrows():
    # 개봉일 기준
    open = datetime.datetime.strftime(row['개봉일'], '%Y-%m-%d')
    director = row['감독'].strip()
    # Top300 중 개봉일 이전 영화의 감독 확인
    directors = movie_resize_ranking[(movie_resize_ranking['개봉일'] < open) & (movie_resize_ranking['감독'] == director)]
    # 해당 감독이 있다면 흥행 여부 표시
    if len(directors) > 0:
        score = directors.index[0]
        if score >= 271:
            score = 1
        elif score >= 241:
            score = 2
        elif score >= 211:
            score = 3
        elif score >= 181:
            score = 4
        elif score >= 151:
            score = 5
        elif score >= 121:
            score = 6
        elif score >= 91:
            score = 7
        elif score >= 61:
            score = 8
        elif score >= 31:
            score = 9
        else:
            score = 10
        movie_final.loc[idx, '감독_흥행'] = score

In [9]:
# 배급사 정수화 코드 
for idx, row in movie_final.iterrows():
    # 개봉일 기준
    open = datetime.datetime.strftime(row['개봉일'], '%Y-%m-%d')
    director = row['배급사'].strip()
    # Top300 중 개봉일 이전 영화의 감독 확인
    directors = movie_resize_ranking[(movie_resize_ranking['개봉일'] < open) & (movie_resize_ranking['배급사'] == director)]
    # 해당 감독이 있다면 흥행 여부 표시
    if len(directors) > 0:
        score = directors.index[0]
        if score >= 271:
            score = 1
        elif score >= 241:
            score = 2
        elif score >= 211:
            score = 3
        elif score >= 181:
            score = 4
        elif score >= 151:
            score = 5
        elif score >= 121:
            score = 6
        elif score >= 91:
            score = 7
        elif score >= 61:
            score = 8
        elif score >= 31:
            score = 9
        else:
            score = 10
        movie_final.loc[idx, '배급사_흥행'] = score

In [10]:
# 배우의 경우 '["배우1", "배우2"]'와 같은 형태이므로 각 값을 뽑아낸다.
# literal_evel : 문자열 그대로 파이썬 명령을 실행시켜주는 함수
# 예시---------------------------------------
# 문자열 : "['문자열1', '문자열2', 22, '33']"
# 리스트 : ['문자열1', '문자열2', 22, '33']
from ast import literal_eval

for idx, row in movie_final.iterrows():
    # 개봉일 기준
    open = datetime.datetime.strftime(row['개봉일'], '%Y-%m-%d')
    # 해당 영화의 배우 확인
    this_actors = literal_eval(row['출연'])
    scores = [0] * len(this_actors)
    # 개봉일 이전 영화의 모든 배우 확인
    for i in range(len(this_actors)):
        actors = movie_resize_ranking.query(f'개봉일 < "{open}"')
        actors = actors.query(
            f'출연.str.contains("{this_actors[i]}")', engine='python')
        if len(actors) > 0:
            score = actors.index[0]
            if score >= 271:
                score = 1
            elif score >= 241:
                score = 2
            elif score >= 211:
                score = 3
            elif score >= 181:
                score = 4
            elif score >= 151:
                score = 5
            elif score >= 121:
                score = 6
            elif score >= 91:
                score = 7
            elif score >= 61:
                score = 8
            elif score >= 31:
                score = 9
            else:
                score = 10
            scores[i] = score

    # 평균/합/최대값 중에 잘 나오는 걸로        
    # 평균값
    # movie_final.loc[idx, '주연배우_흥행'] = round(sum(scores) / len(scores), 1)
    # 합
    movie_final.loc[idx, '주연배우_흥행'] = sum(scores)
    # 최대값
    # movie_final.loc[idx, '주연배우_흥행'] = max(scores)


In [12]:
# 값 확인
print((movie_final['감독_흥행'] == 0).value_counts())
print()
print((movie_final['배급사_흥행'] == 0).value_counts())
print()
print((movie_final['주연배우_흥행'] == 0).value_counts())

True     2801
False     304
Name: 감독_흥행, dtype: int64

False    1599
True     1506
Name: 배급사_흥행, dtype: int64

True     1618
False    1487
Name: 주연배우_흥행, dtype: int64


### movie final 저장

In [16]:
movie_final.drop(['index','Unnamed: 0'],axis=1,inplace=True)

In [17]:
# 변경 될때만 따로 풀어서 사용하시길.(파일생성코드)
# movie_final.to_csv('./movie_final.csv',encoding='utf-8')